In [18]:
from selenium import webdriver
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options

In [21]:
data = pd.read_csv('domain2.csv')

In [30]:
# data = data.head(5)
data

In [31]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument("--disable-javascript")  # Disable JavaScript execution

In [33]:
email_pattern = re.compile("[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}")
domain_pattern = re.compile(
"^(http[s]?:\\/\\/(www\\.)?|ftp:\\/\\/(www\\.)?|www\\.){1}([0-9A-Za-z-\\.@:%_\+~#=]+)+((\\.[a-zA-Z]{2,3})+)(/(.)*)?(\\?(.)*)?",
re.IGNORECASE,
)
image_pattern = re.compile(
"[^\\s]+(.*?)\\.(jpg|jpeg|png|gif|JPG|JPEG|PNG|GIF)$", re.IGNORECASE
)

In [34]:
def scrape_email(domain):
        driver = webdriver.Chrome(options=chrome_options)
        list_emails = []
        attempts = 3
        for attempt in range(attempts):
            try:
                if domain_pattern.match(domain):
                    driver.get(domain)
                else:
                    driver.get("http://" + domain + "/")
                page_source = driver.page_source
                emails = re.findall(email_pattern, page_source)
                for email in emails:
                    if email not in list_emails and image_pattern.match(email) is None:
                        list_emails.append(email)
                
                driver.quit()
                if len(list_emails) == 0:
                    print(f"Email not found in {domain}")
                else:
                    print(f"Email found in {domain} : {list_emails}")
                return list_emails  
            except Exception as e:
                print(f"Attempt{attempt+1}: Error fetching emails for {domain}: {str(e)}")

In [35]:
data['Email'] = data['Domain'].apply(scrape_email)

Email not found in couponannie.com
Email found in thoughtshift.co.uk : ['hello@thoughtshift.co.uk']
Email found in jsfest.com.ua : ['info@jsfest.com.ua', 'finance@jsfest.com.ua']
Email found in zplux.com : ['support@zplux.com']
Email not found in sguru.org


C:\Users\MSI-1\AppData\Local\Temp\ipykernel_6496\2474351608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Email'] = data['Domain'].apply(scrape_email)


In [37]:
data

,Domain,Email
0,couponannie.com,[]
1,thoughtshift.co.uk,[hello@thoughtshift.co.uk]
2,jsfest.com.ua,"[info@jsfest.com.ua, finance@jsfest.com.ua]"
3,zplux.com,[support@zplux.com]
4,sguru.org,[]


In [28]:
filename = 'domain22.csv'
data.to_csv(filename,index = False)